## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,79.63,81,44,15.30,scattered clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,31.87,84,82,1.23,broken clouds
2,2,Mount Holly,US,35.2982,-81.0159,85.10,63,75,10.36,broken clouds
3,3,Mount Isa,AU,-20.7333,139.5000,71.37,64,75,3.44,broken clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,56.32,98,0,5.75,clear sky


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Katsuura,JP,35.1333,140.3000,79.63,81,44,15.30,scattered clouds
2,2,Mount Holly,US,35.2982,-81.0159,85.10,63,75,10.36,broken clouds
5,5,Hithadhoo,MV,-0.6000,73.0833,81.79,71,100,15.48,overcast clouds
6,6,Kankakee,US,41.1500,-87.8667,81.82,61,0,9.22,clear sky
8,8,Avarua,CK,-21.2078,-159.7750,78.85,73,40,18.41,scattered clouds
12,12,Bongor,TD,10.2806,15.3722,76.41,85,89,4.32,overcast clouds
14,14,Touros,BR,-5.1989,-35.4608,75.11,79,0,17.40,clear sky
19,19,Nakhon Phanom,TH,17.3333,104.4167,82.08,96,100,1.83,overcast clouds
20,20,Atuona,PF,-9.8000,-139.0333,76.80,81,29,28.16,light rain
21,21,Jaicos,BR,-7.3592,-41.1378,81.14,31,39,8.08,scattered clouds


In [25]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# N/A

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,79.63,scattered clouds,35.1333,140.3000,
2,Mount Holly,US,85.10,broken clouds,35.2982,-81.0159,
5,Hithadhoo,MV,81.79,overcast clouds,-0.6000,73.0833,
6,Kankakee,US,81.82,clear sky,41.1500,-87.8667,
8,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
12,Bongor,TD,76.41,overcast clouds,10.2806,15.3722,
14,Touros,BR,75.11,clear sky,-5.1989,-35.4608,
19,Nakhon Phanom,TH,82.08,overcast clouds,17.3333,104.4167,
20,Atuona,PF,76.80,light rain,-9.8000,-139.0333,
21,Jaicos,BR,81.14,scattered clouds,-7.3592,-41.1378,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        pass

In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Katsuura,JP,79.63,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
2,Mount Holly,US,85.10,broken clouds,35.2982,-81.0159,Holiday Inn Express & Suites Charlotte Arpt-Be...
5,Hithadhoo,MV,81.79,overcast clouds,-0.6000,73.0833,Scoop Guest House
6,Kankakee,US,81.82,clear sky,41.1500,-87.8667,Fairfield Inn Kankakee Bourbonnais
8,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
12,Bongor,TD,76.41,overcast clouds,10.2806,15.3722,Oryx Palace
14,Touros,BR,75.11,clear sky,-5.1989,-35.4608,INN NEW HORIZON
19,Nakhon Phanom,TH,82.08,overcast clouds,17.3333,104.4167,ป่ายางรีสอร์ท
20,Atuona,PF,76.80,light rain,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
21,Jaicos,BR,81.14,scattered clouds,-7.3592,-41.1378,Hotel Jaicós


In [47]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))